https://medium.com/@EjiroOnose/building-llm-applications-locally-with-langchain-ollama-and-phoenix-46157a27a666

In [1]:
!pip install langchain
!pip install langchain_ollama
!pip install -U langchain-community
!pip install langchain_huggingface
!pip install chromadb
!pip install pypdf
!pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.7/788.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 43.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 34.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 42.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.1/25.1 MB 46.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
!ollama serve &


OSError: Background processes not supported.

In [1]:
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import ollama

In [2]:
# Load documents from a PDF
loader = PyPDFLoader('/Users/sheetalkadam/Documents/passportRenewal/tatkal_adult_reissue_of_passport.pdf')
docs = loader.load()
# Split Documents into Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(docs)


In [3]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/var/folders/r2/w97h36m56qq6nqkfn3rvtxj40000gn/T/ipykernel_6389/2576288788.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
/Users/sheetalkadam/projects/gitrepo/experiment_ai_projects/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
!pip list | grep numpy


numpy                                    1.26.4


In [5]:
# Build a Vector Store (Chroma)
db = Chroma.from_documents(texts, embedding=embeddings)

In [39]:
# From langchain_ollama import ollama
from langchain_ollama.llms import OllamaLLM

# Instantiate the Ollama class directly.
llm = OllamaLLM(model="llama3")

In [40]:
from langchain.prompts import ChatPromptTemplate
system_prompt = (
 "You are an assistant responsible for answering questions and are also bank operations personnel. "
 "Please think carefully about the question and then perform a search. "
 "Use the following retrieved context to answer the question: "
 "Context: {context} "
 "Question: {input} "
 "If you don't know the answer, say you don't know; do not provide irrelevant answers. "
 "Please answer keep your answers concise."
)
prompt = ChatPromptTemplate.from_messages([
 ("system", system_prompt),
 ("human", "{input}")
])

In [41]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})



In [42]:
def format_docs(documents):
 formatted_docs = []
 for doc in documents:
    title = doc.metadata.get('title', 'Unknown Title')
    formatted_docs.append(f"Title: {title}\n\n{doc.page_content}")
 return "\n\n".join(formatted_docs)


In [43]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
 {"context": retriever | format_docs, "input": RunnablePassthrough()}
 | prompt
 | llm
 | StrOutputParser()
)


In [44]:
def RAG(prompt, llm, question, retriever):
 docs = retriever.get_relevant_documents(question)
 formatted_docs = format_docs(docs)
 inputs = {"context": formatted_docs, "input": question}
 chunk = rag_chain.invoke(formatted_docs)
 return chunk

In [46]:
question = "can i apply in tatkal?"
response = RAG(prompt, llm, question, retriever)
print(response)

The document you provided is an adult checklist for a Tatkal passport re-issue application. Here's a breakdown of the different sections and their requirements:

1. MANDATORY DOCUMENTS:
	* Must select the correct jurisdiction online on the Passport Government form.
	* Must choose passport re-issue.
	* Passport government ARN number must match the VFS payment receipt number.
	(DIGITAL SIGN ATURES ARE NOT ACCEPTABLE)
2. ADULT CHECKLIST - TATKAL RE-ISSUE OF PASSPORT:
	* Applicant's name
	* Passport number
	* Email ID
	* Mobile number
	* 6 ANNEURO E - Click here to download (Must be signed by the applicant)
	* Original
	* Copy of the valid status in the USA:
		+ Notarized valid Green Card - FRONT AND BACK
		+ Notarized valid Visa

Please note that the checklist may vary depending on individual circumstances, and it is always best to consult with the Indian embassy or consulate for the most up-to-date requirements.


In [ ]:
question = "can i apply in tatkal?"
response = RAG(prompt, llm, question, retriever)
print(response)

In [47]:
!pip install -qq "arize-phoenix" "litellm" openinference-instrumentation-litellm


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
import os
from phoenix.evals import LiteLLMModel
# Set up the Ollama API base
os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"
# Initialize the model for Arize Phoenix
model = LiteLLMModel(model="ollama/llama2")

In [53]:
from openinference.instrumentation.litellm import LiteLLMInstrumentor
import phoenix as px
from phoenix.otel import register
# Register OpenTelemetry
tp = register()
# Instrument the LLM model
LiteLLMInstrumentor().instrument(tracer_provider=tp)
# Launch the Phoenix app for monitoring
px.launch_app()



Overriding of current TracerProvider is not allowed
Attempting to instrument while already instrumented


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [57]:
model("What documents are required for passport renewal?")



"To renew a passport, you will need to provide certain documents to prove your identity and citizenship. The specific documents required may vary depending on the country or region where you are applying for the passport renewal, but here are some common documents that are typically accepted:\n\n1. Proof of Citizenship: You will need to provide a certified copy of your birth certificate or a naturalization certificate to prove your citizenship.\n2. Valid Passport: You will need to submit your current passport as proof of your identity and eligibility for renewal.\n3. Photo Identification: A valid driver's license, state ID card, or government-issued ID is typically required to provide a photo of yourself for the new passport.\n4. Proof of Name Change (if applicable): If you have changed your name since your last passport was issued, you will need to provide legal documentation of the name change, such as a marriage certificate or court order.\n5. Passport Photo: You will need to provid

In [58]:
px.close_app()

In [ ]:
# Things to do further
# Understand code
# explore the tools- ChromaDB, Arise Phonix